In [133]:
import numpy as np
import pickle
import os
import sys
from sklearn.cluster import KMeans
import myutils

import matplotlib.pyplot as plt
%matplotlib inline

In [134]:
def get_data(subejcts):
    data = {}
    for subj in subjects:
        with open("C:/ASM/DevData/AirPass/genuine_processed/"+subj+".pkl", "rb") as file:
            data[subj] = pickle.load(file)
    return data

In [135]:
passwords = {1:"123456", 2:"abc123", 3:"password"}
subjects = ["abu", "liza"]
subject_count = len(subjects)
data = get_data(subjects)

In [145]:
def DTW_euclidean(a, b, a_indices, b_indices):
    a_six, a_eix = a_indices
    b_six, b_eix = b_indices
    
    n = a_eix - a_six + 1
    m = b_eix - b_six + 1    
        
    D = np.zeros((n+1, m+1)) + 1e10            
    D[0, 0] = 0
    for i in range(1, n+1):
        for j in range(1, m+1):
            d = a[a_six + i-1, 1:4] - b[b_six + j-1, 1:4]
            D[i, j] = np.sum(d*d) + min([D[i-1, j-1], D[i, j-1], D[i-1, j]])
            
    distance = D[n, m]        
    return distance  

In [146]:
def DTW(a, b, a_indices, b_indices):
    a_six, a_eix = a_indices
    b_six, b_eix = b_indices
    
    n = a_eix - a_six + 1
    m = b_eix - b_six + 1    
        
    D = np.zeros((3, n+1, m+1)) + 1e10            
    D[:, 0, 0] = 0
    for i in range(1, n+1):
        for j in range(1, m+1):
            d = np.absolute(a[a_six + i-1, 1:4] - b[b_six + j-1, 1:4])
            for axis in range(3):                
                D[axis, i, j] = d[axis] + min([D[axis, i-1, j-1], D[axis, i, j-1], D[axis, i-1, j]])
            
    distance = D[0, n, m] + D[1, n, m] + D[2, n, m]
    return distance  

In [147]:
def DTW_step(a, b, a_indices, b_indices, step_size):
    
    bmin_ix, b_six, b_eix, bmax_ix = b_indices
    
    res = []
    for s in range(b_six, bmin_ix-1, -step_size):
        e = s + b_eix - b_six
        d = DTW(a, b, a_indices, [s, e])
        res.append(d)
        
    for e in range(b_eix+step_size, bmax_ix+1, step_size):
        s = e - (b_eix - b_six)
        d = DTW(a, b, a_indices, [s, e])
        res.append(d)
    
    return min(res)

In [148]:
def distance(a, b, pw_len, step): # a is the password, b is the template
    npoints = 2*pw_len + 2
    apts = np.round(np.linspace(0, len(a)-1, npoints))
    bpts = np.round(np.linspace(0, len(b)-1, npoints))
    apts = apts.astype(int)
    bpts = bpts.astype(int)
    #print("a, apts:", len(a), apts)
    #print("b, apts:", len(b), bpts)
    #print(len(a), len(b), pw_len, step)
    
    d = 0
    for i in range(npoints-2):
        a_indices = [ apts[i], apts[i+2] ]
        b_indices = [ bpts[max([0, i-1])],  bpts[i], bpts[i+2], bpts[min(i+3, npoints-1)] ]
        #print(a_indices)
        #print(b_indices)
        d += DTW_step(a, b, a_indices, b_indices, step)
        
    return d

In [149]:
def train_model(pws, pw_len, step):
    pw_count = len(pws)
    ds = []    
    for i in range(pw_count):
        for j in range(i+1, pw_count):
            d1 = distance(pws[i], pws[j], pw_len, step)            
            d2 = distance(pws[j], pws[i], pw_len, step)
            print("Distance {}, {} :{}, {}".format(i, j, d1, d2))
            ds.append(d1)
            ds.append(d2)
    
    mu = np.mean(ds)
    sigma = np.std(ds)    
    model = {"pws":pws, "ds":ds, "mu":mu, "sigma":sigma}
    return model
    
    

In [150]:
def test_pw(model, pw, pw_len, step):    
    ds = []
    for p in model["pws"]:
        d1 = distance(p, pw, pw_len, step)  
        d2 = distance(pw, p, pw_len, step)  
        ds.append(d1)
        ds.append(d2)
        
    d = np.mean(ds)
    sigma = (d - model["mu"])/model["sigma"]
    return sigma
    
def test_pws(model, pws, pw_len, step):
    res = []
    for p in pws:
        sigma = test_pw(model, p, pw_len, step)
        print(sigma)
        res.append(sigma)        
    return res

In [151]:
def check_training_pws(pws, pw_len, step):
    pw_count = len(pws)
    max_sigma = -1
    max_index = -1
    for i in range(pw_count):
        pws_new = []
        for j in range(pw_count):
            if j!=i:
                pws_new.append(pws[j])
                
        model = train_model(pws_new, pw_len, step)        
        sigma = test_pw(model, pws[i], pw_len, step)
        print("Password {}>> model mu :{}, model sigma:{}, pw sigma:{}".format(i, model["mu"], model["sigma"], sigma))
        
        if sigma >max_sigma:
            max_sigma = sigma
            max_index = i
            
    return i, max_sigma

In [162]:
pw_no = 1


pws1 = data["abu"][pw_no]
pws2 = data["liza"][pw_no]
pw_len = len(passwords[pw_no])

step = 5
#i, max_sigma = check_training_pws(pws[:5], pw_len = len(passwords[pw_no]), step = step)
model = train_model(pws2[:5], pw_len, step)
print("Model mu:{}, sigma:{}".format(model["mu"], model["sigma"]))


Distance 0, 1 :679.2606634703977, 632.798931967999
Distance 0, 2 :561.7248049424251, 553.5677426477029
Distance 0, 3 :554.3274778595799, 554.5399244918108
Distance 0, 4 :587.9318810914848, 571.2806992506761
Distance 1, 2 :688.0087587301209, 678.5918132198935
Distance 1, 3 :655.5406261585858, 647.6287569281163
Distance 1, 4 :676.3501720174122, 645.85899127454
Distance 2, 3 :553.0178641041709, 577.2276312028213
Distance 2, 4 :558.2998001212684, 553.2842318076348
Distance 3, 4 :554.2479655495717, 541.9099066173272
Model mu:601.269932172677, sigma:52.62722587432579


In [163]:
res1 = test_pws(model, pws2[5:], pw_len, step)
print(res1)

res2 = test_pws(model, pws1, pw_len, step)
print(res2)


1.67964957359
1.69296750622
2.71116055176
0.533482489724
1.10270631062
[1.6796495735890362, 1.6929675062207528, 2.7111605517595394, 0.5334824897237519, 1.1027063106185968]
4.3931672715
3.51232025471
4.36335238529
4.18051325881
4.9387692928
8.65722318983
4.14597846008
4.80210156997
4.02736924974
4.37417974212
[4.3931672715000376, 3.5123202547126953, 4.3633523852900282, 4.180513258810941, 4.9387692927975229, 8.6572231898303631, 4.1459784600839589, 4.8021015699709215, 4.0273692497377027, 4.3741797421210293]
